## *Load Libraries*

In [1]:
import numpy as np
import joblib
import json
import torch.nn as nn
import torch.nn.functional as F
import torch
import warnings

In [2]:
warnings.filterwarnings('ignore')

## *Load Scaler Objects*

In [3]:
def scaler_inputs(x):
    url = "C:\\Users\\PC\\Desktop\\Ciencias de datos\\Proyectos\\Criaglist\\tools\\scaler_inputs.file"
    scaler = joblib.load(url)
    return scaler.transform(x)

def scaler_target(pred):
    url = "C:\\Users\\PC\\Desktop\\Ciencias de datos\\Proyectos\\Criaglist\\tools\\scaler_target.file"
    scaler = joblib.load(url)
    return scaler.inverse_transform(pred)

We load the data scalers for the numeric variables since they were thus trained and as a consequence the model will return the standardized data for which we import another object to return the data to its natural state.

## *Load JSON File*

In [4]:
def json_file():
    url = "C:\\Users\\PC\\Desktop\\Ciencias de datos\\Proyectos\\Criaglist\\tools\\columns.json"
    with open(url) as F:
        
        json_file = json.loads(F.read())
        json_file = np.asarray(json_file['column'])

    return json_file

We load the JSON file to favor the OHE transformation for the qualitative categorical variables.

In [5]:
columns = json_file()

In [6]:
np.where('mustang'==columns)[0][0]

4240

We will use said file to use it to return the respective index of the variable.

## *Load Model*

In [7]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()

        self.fc1 = nn.Linear(in_features = 6650,out_features = 128)
        self.fc2 = nn.Linear(in_features = 128,out_features = 64)
        self.fc3 = nn.Linear(in_features= 64,out_features=32)
        self.output = nn.Linear(in_features = 32,out_features = 1)
    
    def forward(self,x):
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.dropout(x,0.3)
    
        
        return self.output(x)

We built the same architecture of the neural network.

In [8]:
url = "C:\\Users\\PC\\Desktop\\Ciencias de datos\\Proyectos\\Criaglist\\model\\craiglist_model.pth"

In [9]:
model = Model()

In [10]:
model.load_state_dict(torch.load(url))

<All keys matched successfully>

We use the network we created as a container to pass the already trained model file.

## *Preprocessing of new data*

In [11]:
def preprocessing(year,
            condition,
            cylinders,
            odometer,
            manufacturer,
            model,
            fuel,
            title_status,
            transmission,
            drive,
            type_body,
            paint_color,
           premium):
    
    
    zeros = np.zeros(len(columns))
  
    

    manufacturer_idx = np.where(manufacturer == columns)[0][0]
    model_idx = np.where(model == columns)[0][0]
    fuel_idx = np.where(fuel == columns)[0][0]
    title_status_idx = np.where(title_status == columns)[0][0]
    
    transmission_idx = np.where(transmission == columns)[0][0]
    drive_idx = np.where(drive == columns)[0][0]
    type_body_idx = np.where(type_body == columns)[0][0]
    color_idx = np.where(paint_color == columns)[0][0]
    
    if manufacturer_idx >= 0:
        zeros[manufacturer_idx] = 1
    
    if model_idx >=0:
        zeros[model_idx] = 1
        
    if fuel_idx >=0:
        zeros[fuel_idx] = 1
        
    if title_status_idx>=0:
        zeros[title_status_idx] = 1
        
    if transmission_idx >=0:
        zeros[transmission_idx] = 1
        
    if drive_idx>=0:
        zeros[drive_idx] = 1
        
    if type_body_idx>=0:
        zeros[type_body_idx] = 1
        
    if color_idx>=0:
        zeros[color_idx] = 1
#        ['year','condition','odometer','cylinders']

    condition_dict = {'salvage':0,
                      'fair':1,
                      'good':2,
                      'excellent':3,
                     'like new':4}
    
    condition = condition_dict[condition]
        
    scaler_data = scaler_inputs([[year,condition,odometer,cylinders]]).flatten()
    zeros[0] = scaler_data[0]
    zeros[1] = scaler_data[1]
    zeros[2] = scaler_data[3]
    zeros[3] = scaler_data[2]
    
    
    zeros[4] = np.where(premium=='yes',1,0)
    zeros = torch.from_numpy(zeros).float()
  #pred = model(zeros)
   

    return zeros
  

We create a function to preprocess the new data and it will return the transformed data in Tensor format.

In [12]:
 preprocessing(2021,
                  'like new',
                  6,
                   17000,
                  'gmc',
                  'sierra',
                  'diesel',
                  'clean',
                  'automatic',
                  '4wd',
                  'pickup',
                  'white',
                  'yes')

tensor([1.8480, 1.9990, 0.0432,  ..., 0.0000, 1.0000, 0.0000])

## *Predict new data*

In [14]:
def predict(newdata):
    
    pred = model(newdata).detach().reshape(-1,1)
    pred = scaler_target(pred)
    pred = np.round(pred,2)
    
    return pred

###  *2016 Nissan Sentra*

<img src = "https://images.kavak.services/images/225657/EXTERIOR-frontSidePilotNear-1670508918891.jpeg?d=540x310">

In [15]:
newdata = preprocessing(2016,
                  'good',
                  4,
                   96000,
                  'nissan',
                  'sentra',
                  'gas',
                  'clean',
                  'automatic',
                  'fwd',
                  'sedan',
                  'white',
                  'no')

pred = predict(newdata)
print(f'Estimated price for a 2016 Nissan Sentra: $ {pred} USD')

Estimated price for a 2016 Nissan Sentra: $ [[5276.57]] USD


### *2021 BMW M4*

<img src = "https://i.ytimg.com/vi/rNHgzipoOZE/maxresdefault.jpg">

In [16]:
newdata = preprocessing(2021,
                  'like new',
                  6,
                   8000,
                  'bmw',
                  'm4',
                  'gas',
                  'clean',
                  'automatic',
                  'rwd',
                  'coupe',
                  'black',
                  'yes')


pred = predict(newdata)
print(f'Estimated price for a 2021 BMW M4: $ {pred} USD')

Estimated price for a 2021 BMW M4: $ [[77477.2]] USD


### *2016 Dodge Charger Police*

<img src = "https://http2.mlstatic.com/D_NQ_NP_978772-MLM51938341807_102022-O.webp">

In [17]:
newdata = preprocessing(2016,
                  'excellent',
                  6,
                   40000,
                  'dodge',
                  'charger police',
                  'gas',
                  'clean',
                  'automatic',
                  'rwd',
                  'sedan',
                  'red',
                  'no')


pred = predict(newdata)
print(f'Estimated price for a 2016 Dodge Charger Police: $ {pred} USD')

Estimated price for a 2016 Dodge Charger Police: $ [[20292.99]] USD


### *2001 Ford Mustang*

<img src = "https://http2.mlstatic.com/D_NQ_NP_680649-MLM52973166807_122022-O.webp">

In [18]:
newdata = preprocessing(2001,
                      'good',
                      6,
                       90000,
                      'ford',
                      'mustang',
                      'gas',
                      'rebuilt',
                      'manual',
                      'rwd',
                      'convertible',
                      'silver',
                      'no')

pred = predict(newdata)
print(f'Estimated price for a 2001 Ford Mustang: $ {pred} USD')

Estimated price for a 2001 Ford Mustang: $ [[7119.45]] USD


### *2020 Honda Accord Touring*

<img src = "https://img.remediosdigitales.com/a7a3b1/honda-accord-2021-precio-mexico_/1366_2000.jpg">

In [19]:
newdata = preprocessing(2020,
                      'excellent',
                      4,
                       71000,
                      'honda',
                      'accord touring',
                      'gas',
                      'clean',
                      'automatic',
                      'fwd',
                      'coupe',
                      'grey',
                      'no')

pred = predict(newdata)
print(f'Estimated price for a 2020 Honda Touring: $ {pred} USD')

Estimated price for a 2020 Honda Touring: $ [[24385.45]] USD


### *2002 Ford Ranger*

<img src = "https://1830262577.rsc.cdn77.org/files/1FTYR14U32TA01138/IAAI/31035473/photo/photo_1.jpg">

In [20]:
newdata = preprocessing(2002,
                      'good',
                      4,
                       150000,
                      'ford',
                      'ranger',
                      'gas',
                      'clean',
                      'manual',
                      '4wd',
                      'truck',
                      'blue',
                      'no')

pred = predict(newdata)
print(f'Estimated price for a 2002 Ford Ranger: $ {pred} USD')

Estimated price for a 2002 Ford Ranger: $ [[1471.18]] USD


### *2012 Chevrolet Suburban*

<img src = "https://http2.mlstatic.com/D_NQ_NP_798251-MLM52446835899_112022-W.jpg">

In [21]:
newdata = preprocessing(2012,
                      'excellent',
                      8,
                       77000,
                      'chevrolet',
                      'suburban',
                      'gas',
                      'clean',
                      'automatic',
                      '4wd',
                      'suv',
                      'black',
                      'no')

pred = predict(newdata)
print(f'Estimated price for a 2012 Chevrolet Suburban: $ {pred} USD')

Estimated price for a 2012 Chevrolet Suburban: $ [[28584.33]] USD


### *Mazda CX-5*

<img src = "https://http2.mlstatic.com/D_NQ_NP_661197-MLM52252354442_112022-O.webp">

In [27]:
newdata = preprocessing(2021,
                      'like new',
                      6,
                       13000,
                      'mazda',
                      'cx-5',
                      'gas',
                      'clean',
                      'automatic',
                      'fwd',
                      'suv',
                      'white',
                      'no')

pred = predict(newdata)
print(f'Estimated price for a Mazda CX-5: $ {pred} USD')

Estimated price for a Mazda CX-5: $ [[33127.92]] USD


### *2014 GMC Sierra 1500*

<img src = "https://http2.mlstatic.com/D_NQ_NP_709493-MLM52842367890_122022-O.webp">

In [28]:
 newdata = preprocessing(2014,
                  'good',
                  8,
                   80000,
                  'gmc',
                  'sierra',
                  'gas',
                  'clean',
                  'automatic',
                  '4wd',
                  'pickup',
                  'white',
                  'yes')

pred = predict(newdata)
print(f'Estimated price for a 2014 GMC Sierra 1500: $ {pred} USD')

Estimated price for a 2014 GMC Sierra 1500: $ [[25526.71]] USD


###  *2001  Toyota Tacoma*

<img src = "https://http2.mlstatic.com/D_NQ_NP_738407-MLM52815866483_122022-O.webp">

In [29]:
newdata =  preprocessing(2001,
                  'good',
                  4,
                   67000,
                  'toyota',
                  'tacoma',
                  'gas',
                  'rebuilt',
                  'manual',
                  '4wd',
                  'pickup',
                  'red',
                  'no')

pred = predict(newdata)
print(f'Estimated price for a 2001 Toyota Tacoma: $ {pred} USD')

Estimated price for a 2001 Toyota Tacoma: $ [[8136.62]] USD
